In [35]:
import json
from pprint import pprint
from analysis.gender import gender, gender_special
from analysis.utils import get_people_mentioned, get_gender

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def process_coref_chain(coref_id, coref_chain):
    # Get the unique representative element
    rep_elem = next(coref for coref in coref_chain if coref['isRepresentativeMention'])
    to_print = [(elem['text'], elem['type'], elem['number'], elem['gender'], elem['sentNum']) for elem in coref_chain
                if elem['animacy'] == 'ANIMATE']
    if len(to_print):
        print len(coref_chain), to_print

In [15]:
def load_tc_data(year, month, folder='annotated/'):
    tc_data = {}
    with open('{}techcrunch_annotated_{}_{}.tsv'.format(folder, year, month), 'r') as tc_f:
        for line in tc_f:
            link, data, corenlp = line.strip().split('\t')
            tc_data[link] = {'data': json.loads(data), 'corenlp': json.loads(corenlp)}
    
    return tc_data

In [8]:
def get_mentions_counts(data):
    all_counts = {'MALE': {'MALE': 0, 'FEMALE': 0}, 'FEMALE': {'MALE': 0, 'FEMALE': 0}, 'UNK': {'MALE': 0, 'FEMALE': 0}}
    for link, data in data.iteritems():
        try:
            gend = get_gender(data['author'])
            if type(gend) is str:
                gend = gend.upper()
            else:
                gend = 'UNK'
            if not gend:
                continue
            counts = all_counts[gend]
            sentences = data['corenlp']['sentences']
            corefs = data['corenlp']['corefs']
            people_mentioned = process_sentences(sentences)
            pm_to_gender = {pm: None for pm in people_mentioned}
            for c_id, coref_chain in corefs.iteritems():
                rep_elem = next(coref for coref in coref_chain if coref['isRepresentativeMention'])
                elem_text = set(rep_elem['text'].split())
                for pm in pm_to_gender:
                    if len(set.intersection(elem_text, set(pm))) > 0:
                        if rep_elem['gender'] in ['MALE', 'FEMALE']:
                            pm_to_gender[pm] = rep_elem['gender']
            for gender in ['MALE', 'FEMALE']:
                counts[gender] += len([sp for sp in pm_to_gender.values() if sp == gender])
        except:
            continue
    return all_counts

In [8]:
for year in range(2009, 2017):
    with open('data/annotated_old/techcrunch_annotated_{}.json'.format(year), 'r') as tc_f:
        data = json.load(tc_f)
        print year, get_mentions_counts(data)

2009 {'UNK': {'MALE': 4006, 'FEMALE': 440}, 'MALE': {'MALE': 9721, 'FEMALE': 1066}, 'FEMALE': {'MALE': 2520, 'FEMALE': 301}}
2010 {'UNK': {'MALE': 5241, 'FEMALE': 567}, 'MALE': {'MALE': 10354, 'FEMALE': 1047}, 'FEMALE': {'MALE': 4902, 'FEMALE': 537}}
2011 {'UNK': {'MALE': 5147, 'FEMALE': 544}, 'MALE': {'MALE': 9420, 'FEMALE': 970}, 'FEMALE': {'MALE': 8884, 'FEMALE': 1025}}
2012 {'UNK': {'MALE': 4406, 'FEMALE': 441}, 'MALE': {'MALE': 10956, 'FEMALE': 1223}, 'FEMALE': {'MALE': 7407, 'FEMALE': 925}}
2013 {'UNK': {'MALE': 3397, 'FEMALE': 346}, 'MALE': {'MALE': 14846, 'FEMALE': 1615}, 'FEMALE': {'MALE': 9682, 'FEMALE': 1232}}
2014 {'UNK': {'MALE': 954, 'FEMALE': 121}, 'MALE': {'MALE': 14318, 'FEMALE': 1582}, 'FEMALE': {'MALE': 7363, 'FEMALE': 1210}}
2015 {'UNK': {'MALE': 614, 'FEMALE': 72}, 'MALE': {'MALE': 13075, 'FEMALE': 1553}, 'FEMALE': {'MALE': 7555, 'FEMALE': 1399}}
2016 {'UNK': {'MALE': 274, 'FEMALE': 44}, 'MALE': {'MALE': 7386, 'FEMALE': 881}, 'FEMALE': {'MALE': 4755, 'FEMALE': 950}

In [9]:
MIN_NUM = 623 + 6
MAX_NUM = MIN_NUM + 1
curr_idx = 0
for link, data in tc_2016_data.iteritems():
    print data['author'], get_gender(data['author'])
    continue
    curr_idx += 1
    if curr_idx < MIN_NUM:
        continue
    if curr_idx >= MAX_NUM:
        break    
    print link
    corefs = data['corenlp']['corefs']
    sentences = data['corenlp']['sentences']
    # print sentences[0].keys()
    # pprint(sentences[5]['tokens'])
    print process_sentences(sentences)
    for c_id, coref in corefs.iteritems():
        process_coref_chain(c_id, coref)

NameError: name 'tc_2016_data' is not defined

In [17]:
tc_data = load_tc_data(2016, 1)

In [71]:
MIN_NUM = 31
MAX_NUM = MIN_NUM + 15
curr_idx = 0
for link, values in tc_data.iteritems():
    curr_idx += 1
    if curr_idx < MIN_NUM:
        continue
    if curr_idx > MAX_NUM:
        break

    data = values['data']
    corenlp = values['corenlp']
    print link
    corefs = corenlp['corefs']
    # pprint(corefs)
    sentences = corenlp['sentences']
    # pprint(corefs)
    # pprint(sentences[4])
    # print sentences[0].keys()
    # pprint(sentences[5]['tokens'])
    print get_people_mentioned(sentences, corefs, include_gender=True)
    #for c_id, coref in corefs.iteritems():
    #    process_coref_chain(c_id, coref)

https://techcrunch.com/2016/01/31/the-league-vs-stanford/
{u'Amanda Bradford': (5, (u'FEMALE', 'COREF'))}
https://techcrunch.com/2016/01/20/with-brave-software-javascripts-inventor-is-building-a-browser-for-the-ad-blocked-future/
{u'Brendan Eich': (8, (u'MALE', 'COREF'))}
https://techcrunch.com/2016/01/06/stereolabss-depth-sensing-camera-helps-robots-drones-and-cars-see/
{u'Cecile Schmollgruber': (1, (u'FEMALE', 'COREF')), u'Stereolabs': (1, (None, None)), u'Stereolabsthen': (3, (None, None))}
https://techcrunch.com/2016/01/07/no-more-ballparks-oculus-palmer-luckey-admits-screwing-up-rift-price-point-messaging/
{u'Tom Kerriganwrote': (8, ('male', 'NAME_ONLY')), u'Nate Mitchell': (1, (None, None)), u'Palmer Luckey': (1, (u'MALE', 'COREF'))}
https://techcrunch.com/2016/01/09/how-startups-are-solving-a-decades-old-problem-in-education/
{u'Roshan Choxi': (1, ('male', 'NAME_ONLY')), u'Socrates': (1, (None, None)), u'Udemy': (1, (None, None)), u'Udacity': (1, (None, None)), u'Thinkful': (1, 